In [0]:
api_key = "PWEUCUXA4575GXRN"
api_secret = "SYRj4UPfB0qXJcYL32B+9FZYgT+SPsSlSyLA3Ww26v7NRxxLxBSdgP6A5Nqq842j"

In [0]:
dbutils.fs.ls('/FileStore/tables/')

Out[22]: [FileInfo(path='dbfs:/FileStore/tables/1_registered_users_1.csv', name='1_registered_users_1.csv', size=266, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/1_registered_users_2.csv', name='1_registered_users_2.csv', size=263, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/2_user_info_1.json', name='2_user_info_1.json', size=2913, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/2_user_info_2.json', name='2_user_info_2.json', size=2482, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/3_bpm_1.json', name='3_bpm_1.json', size=39990667, modificationTime=1706195982000),
 FileInfo(path='dbfs:/FileStore/tables/3_bpm_2.json', name='3_bpm_2.json', size=39994963, modificationTime=1706195982000),
 FileInfo(path='dbfs:/FileStore/tables/4_workout_1.json', name='4_workout_1.json', size=3319, modificationTime=1706195983000),
 FileInfo(path='dbfs:/FileStore/tables/4_workout_2.json', name='4_workou

In [0]:

from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as f


In [0]:
spark = SparkSession.builder \
        .config("spark.master", "Local[*]") \
        .config("spark.app.name","pushing") \
        .config("spark.sql.catalogImplementation" , 'hive') \
        .getOrCreate()

In [0]:
print(spark.conf.get("spark.app.name"))
print(spark.sparkContext.getConf().toDebugString())

pushing
libraryDownload.sleepIntervalSeconds=5
libraryDownload.timeoutSeconds=180
spark.akka.frameSize=256
spark.app.id=local-1706340395296
spark.app.name=Databricks Shell
spark.app.startTime=1706340385329
spark.cleaner.referenceTracking.blocking=false
spark.databricks.acl.client=com.databricks.spark.sql.acl.client.SparkSqlAclClient
spark.databricks.acl.provider=com.databricks.sql.acl.ReflectionBackedAclProvider
spark.databricks.acl.scim.client=com.databricks.spark.sql.acl.client.DriverToWebappScimClient
spark.databricks.automl.serviceEnabled=true
spark.databricks.autotune.maintenance.client.classname=com.databricks.maintenanceautocompute.MACClientImpl
spark.databricks.cloudProvider=AWS
spark.databricks.cloudfetch.hasRegionSupport=true
spark.databricks.cloudfetch.requestDownloadUrlsWithHeaders=*********(redacted)
spark.databricks.cloudfetch.requesterClassName=*********(redacted)
spark.databricks.clusterSource=UI
spark.databricks.clusterUsageTags.attribute_tag_budget=
spark.databricks.c

In [0]:
df_registered_users = spark.read.format("csv").option("inferSchema","true").option("header","true").load("/FileStore/tables/1_registered_users_*")
display(df_registered_users)
print(df_registered_users.printSchema())
df_key_value = df_registered_users.select(f.col('user_id').cast('string').alias('key'),f.to_json(f.struct("*")).alias('value'))
df_key_value.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "pkc-4r087.us-west2.gcp.confluent.cloud:9092") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(api_key, api_secret)) \
        .option("kafka.ssl.endpoint.identification.algorithm", "https") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("topic","registered_user_topic") \
        .save()

user_id,device_id,mac_address,registration_timestamp
12474,118440,4c:c5:9f:cb:13:bd,1678451168
12140,141687,ae:ec:f6:48:ca:f7,1678451529
13559,109290,36:1f:d9:d3:e8:0d,1678451631
11745,190960,14:cd:d6:db:70:f6,1678451681
12227,114131,57:24:ac:8c:75:ea,1678452028
14633,175406,1d:69:69:75:d0:aa,1678538830
14508,102558,df:f9:dc:5e:e2:a8,1678539025
13937,143442,dd:96:be:e9:1e:f4,1678539037
14232,172965,dd:45:d2:37:a8:0e,1678539087
15149,177966,de:c0:cd:a7:71:f4,1678539102


root
 |-- user_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- mac_address: string (nullable = true)
 |-- registration_timestamp: integer (nullable = true)

None


In [0]:
df_registered_users = spark.read.format("csv").option("inferSchema","true").option("header","true").load("/FileStore/tables/5_gym_logins_*")
display(df_registered_users)
print(df_registered_users.printSchema())
df_key_value = df_registered_users.select(f.col('mac_address').cast('string').alias('key'),f.to_json(f.struct("*")).alias('value'))
df_key_value.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "pkc-4r087.us-west2.gcp.confluent.cloud:9092") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(api_key, api_secret)) \
        .option("kafka.ssl.endpoint.identification.algorithm", "https") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("topic","gym_logins_topic") \
        .save()

mac_address,gym,login,logout
4c:c5:9f:cb:13:bd,5,1678521600,1678526100
ae:ec:f6:48:ca:f7,1,1678522500,1678525200
36:1f:d9:d3:e8:0d,3,1678522500,1678527000
14:cd:d6:db:70:f6,5,1678523400,1678527600
57:24:ac:8c:75:ea,1,1678524000,1678528500
36:1f:d9:d3:e8:0d,3,1678561200,1678564800
14:cd:d6:db:70:f6,5,1678562400,1678565700
57:24:ac:8c:75:ea,5,1678562880,1678567200
1d:69:69:75:d0:aa,1,1678608000,1678611000
df:f9:dc:5e:e2:a8,1,1678608000,1678611600


root
 |-- mac_address: string (nullable = true)
 |-- gym: integer (nullable = true)
 |-- login: integer (nullable = true)
 |-- logout: integer (nullable = true)

None


In [0]:
df_registered_users = spark.read.format("json").option("inferSchema","true").option("header","true").load("/FileStore/tables/3_bpm_*").select('value.*')
display(df_registered_users)
print(df_registered_users.printSchema())
df_key_value = df_registered_users.select(f.col('device_id').cast('string').alias('key'),f.to_json(f.struct("*")).alias('value'))
df_key_value.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "pkc-4r087.us-west2.gcp.confluent.cloud:9092") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(api_key, api_secret)) \
        .option("kafka.ssl.endpoint.identification.algorithm", "https") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("topic","bpm_stream_topic") \
        .save()

device_id,heartrate,time
118440,41.21897868284929,1678410000
118440,46.95949313912609,1678410001
118440,84.39302961093657,1678410002
118440,87.4902278952101,1678410003
118440,76.16501196545016,1678410004
118440,42.46430112225653,1678410005
118440,46.989738006342094,1678410006
118440,93.72495479825955,1678410007
118440,85.67159141451009,1678410008
118440,80.44560147347363,1678410009


root
 |-- device_id: long (nullable = true)
 |-- heartrate: double (nullable = true)
 |-- time: long (nullable = true)

None


In [0]:
df_registered_users = spark.read.format("json").option("inferSchema","true").option("header","true").load("/FileStore/tables/4_workout*")
schema = "user_id long , workout_id int , timestamp double , action string, session_id int"
df_registered_users = df_registered_users.select(f.from_json('value',schema).alias('value')).select('value.*')
display(df_registered_users)
print(df_registered_users.printSchema())
df_key_value = df_registered_users.select(f.col('user_id').cast('string').alias('key'),f.to_json(f.struct("*")).alias('value'))
df_key_value.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "pkc-4r087.us-west2.gcp.confluent.cloud:9092") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(api_key, api_secret)) \
        .option("kafka.ssl.endpoint.identification.algorithm", "https") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("topic","work_out_session_topic") \
        .save()

user_id,workout_id,timestamp,action,session_id
12474,1,1.6785219E9,start,1
12140,1,1.6785228E9,start,1
13559,1,1.6785228E9,start,1
11745,1,1.6785237E9,start,1
12227,1,1.6785243E9,start,1
12140,1,1.6785249E9,stop,1
12474,1,1.6785258E9,stop,1
13559,1,1.6785267E9,stop,1
11745,1,1.6785273E9,stop,1
12227,1,1.6785282E9,stop,1


root
 |-- user_id: long (nullable = true)
 |-- workout_id: integer (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- action: string (nullable = true)
 |-- session_id: integer (nullable = true)

None


In [0]:
df_registered_users = spark.read.format("json").option("inferSchema","true").option("header","true").load("/FileStore/tables/2_user_info_*")
schema = "user_id long, update_type string, timestamp double, dob string, sex string, gender string, first_name string, last_name string, address struct<street_address:string, city:string, state:string, zip:long>"
df_registered_users = df_registered_users.select(f.from_json("value",schema).alias('value')).select('value.*')
display(df_registered_users)
df_key_value = df_registered_users.select(f.to_json(f.struct("*")).alias('value'))
df_key_value.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "pkc-4r087.us-west2.gcp.confluent.cloud:9092") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(api_key, api_secret)) \
        .option("kafka.ssl.endpoint.identification.algorithm", "https") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("topic","user_info_topic") \
        .save()

user_id,update_type,timestamp,dob,sex,gender,first_name,last_name,address
12474,new,1.678451168E9,07/25/1939,M,M,Matthew,Phillips,"List(02648 Wilkins Cliffs Suite 998, San Fernando, CA, 91340)"
12140,new,1.678451529E9,02/02/1999,M,M,Robert,Clark,"List(68994 Steven Vista, Pearblossom, CA, 93553)"
12140,update,1.678451625E9,02/02/1999,M,M,Robert,Castillo,"List(68994 Steven Vista, Pearblossom, CA, 93553)"
13559,new,1.678451631E9,03/06/1980,F,F,Victoria,Smith,"List(88788 Dawson Lodge, Los Angeles, CA, 90065)"
13559,update,1.67845165E9,03/06/1980,F,F,Victoria,Smith,"List(634 Acevedo Mountain, Santa Monica, CA, 90405)"
11745,new,1.678451681E9,06/29/1955,F,F,Shannon,Reyes,"List(3105 Bowers Expressway, Long Beach, CA, 90808)"
12227,new,1.678452028E9,12/11/1949,F,F,Courtney,Sheppard,"List(47754 Angela Plaza Apt. 135, Los Angeles, CA, 90010)"
14633,new,1.67853883E9,09/04/1997,F,F,Hannah,Fuller,"List(81346 Obrien Streets, Gardena, CA, 90249)"
14508,new,1.678539025E9,01/28/1936,M,M,Justin,Eaton,"List(04952 Lori Plain, Sierra Madre, CA, 91024)"
13937,new,1.678539037E9,04/26/1982,M,M,Matthew,Johnson,"List(9231 Edward Throughway Suite 072, Toluca Lake, CA, 91610)"
